## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import os
from dotenv import load_dotenv
load_dotenv()
g_key = os.getenv("GOOGLE_MAPS_API")

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bathsheba,BB,13.2167,-59.5167,88.41,78,40,19.57,scattered clouds
1,1,Da Nang,VN,16.0678,108.2208,80.58,78,20,2.30,few clouds
2,2,Chokurdakh,RU,70.6333,147.9167,47.68,52,100,9.69,overcast clouds
3,3,New Norfolk,AU,-42.7826,147.0587,40.50,83,7,1.01,clear sky
4,4,Castro,BR,-24.7911,-50.0119,72.18,48,100,10.31,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
mintemp, maxtemp = int(input('Enter a min temp: ')), int(input('Enter a max temp: '))

Enter a min temp: 60
Enter a max temp: 80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df_temp = city_data_df.loc[(city_data_df['Max Temp'] >= mintemp) & (city_data_df['Max Temp'] <= maxtemp)]
city_data_df_temp.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Castro,BR,-24.7911,-50.0119,72.18,48,100,10.31,overcast clouds
6,6,Strelka,RU,58.0728,93.0344,62.87,77,100,3.80,overcast clouds
7,7,Thompson,CA,55.7435,-97.8558,73.56,40,20,8.05,few clouds
8,8,Zambezi,ZM,-13.5432,23.1047,60.96,36,32,5.14,scattered clouds
11,11,Souillac,MU,-20.5167,57.5167,70.18,78,20,10.36,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
city_data_df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 309 entries, 4 to 653
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           309 non-null    int64  
 1   City                 309 non-null    object 
 2   Country              309 non-null    object 
 3   Lat                  309 non-null    float64
 4   Lng                  309 non-null    float64
 5   Max Temp             309 non-null    float64
 6   Humidity             309 non-null    int64  
 7   Cloudiness           309 non-null    int64  
 8   Wind Speed           309 non-null    float64
 9   Current Description  309 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 26.6+ KB


In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
city_clean = city_data_df_temp.dropna(axis="index", how="any")
city_clean.head() 

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Castro,BR,-24.7911,-50.0119,72.18,48,100,10.31,overcast clouds
6,6,Strelka,RU,58.0728,93.0344,62.87,77,100,3.80,overcast clouds
7,7,Thompson,CA,55.7435,-97.8558,73.56,40,20,8.05,few clouds
8,8,Zambezi,ZM,-13.5432,23.1047,60.96,36,32,5.14,scattered clouds
11,11,Souillac,MU,-20.5167,57.5167,70.18,78,20,10.36,few clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_clean[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Castro,BR,72.18,overcast clouds,-24.7911,-50.0119,
6,Strelka,RU,62.87,overcast clouds,58.0728,93.0344,
7,Thompson,CA,73.56,few clouds,55.7435,-97.8558,
8,Zambezi,ZM,60.96,scattered clouds,-13.5432,23.1047,
11,Souillac,MU,70.18,few clouds,-20.5167,57.5167,
13,Alnashi,RU,66.36,overcast clouds,56.1874,52.4792,
18,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,
20,Yongan,CN,64.13,clear sky,39.6957,113.6871,
22,Zhigansk,RU,65.10,overcast clouds,66.7697,123.3711,
23,Rikitea,PF,73.56,overcast clouds,-23.1203,-134.9692,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel = hotel_df.dropna(axis="index", how="any")
clean_hotel.head(10) 

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Castro,BR,72.18,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
6,Strelka,RU,62.87,overcast clouds,58.0728,93.0344,"Gostinitsa ""Angara"""
7,Thompson,CA,73.56,few clouds,55.7435,-97.8558,Burntwood Hotel
8,Zambezi,ZM,60.96,scattered clouds,-13.5432,23.1047,Royal Kutachika Lodge
11,Souillac,MU,70.18,few clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
13,Alnashi,RU,66.36,overcast clouds,56.1874,52.4792,"Motel' ""Voyazh"""
18,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
20,Yongan,CN,64.13,clear sky,39.6957,113.6871,Beiyue International Hotel
22,Zhigansk,RU,65.10,overcast clouds,66.7697,123.3711,Gostinichnyy Kompleks Aylgy
23,Rikitea,PF,73.56,overcast clouds,-23.1203,-134.9692,People ThankYou


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))